# Importing and Tidying data

In [1]:
import pandas as pd
import numpy as np

### Modifying HTML code

The given data is in a large table on the Winston's Lab website. This table is almost formatted perfectly, but there are a few columns that have nothing but pictures. I need to somehow convert these images to a string that I can use.

This function will basically replace the image code with the title of the image.

In [2]:
#convert img entries into words to be used
def fix_html_img(in_file_name, out_file_name):
    with open(in_file_name, 'r') as in_file:
        with open(out_file_name, 'w') as out_file:
            for line in in_file:
                string = ''
                if "img" in line and "title" in line:
                    line_sep = line.split('\"')
                    if "class" in line:
                        string = string + ', ' + line_sep[1]
                        string = string + ' ' + line_sep[5]
                    else:
                        string = string + ', ' + line_sep[3]
                else:
                    string = line
                out_file.write(string)
            

Now I need to test to make sure that it works as intended.

In [3]:
#test file 1
test_in = 'untitled3.txt'
test_out = 'test.out'
fix_html_img(test_in, test_out)
with open(test_out, 'r') as file:
    lines = file.readlines()
    assert(len(lines) == 4)
    assert(lines[0] == '<td class="table-count table-basic">\n')
    assert(lines[1] == 'B</td>\n')
    assert(lines[2] == '<td class="table-death-sequence table-basic">\n')
    assert(lines[3] == ', red Winston, blue Zenyatta, blue Winston, blue Mccree, blue D.Va, blue Genji, blue Mccree')

In [4]:
#test file 2
test_in = 'untitled4.txt'
test_out = 'test2.out'
fix_html_img(test_in, test_out)
with open(test_out, 'r') as file:
    lines = file.readlines()
    assert(len(lines) == 5)
    assert(lines[0] == '</td>\n')
    assert(lines[1] == '<td class="table-heroes ults table-adv" style="display: none;">\n')
    assert(lines[2] == ', Mccree, Mercy</td>\n')
    assert(lines[3] == '<td class="table-heroes ults table-adv" style="display: none;">\n')
    assert(lines[4] == ', Mercy, Tracer, Genji</td>')

Ready for the real transformation.

In [5]:
real_in = 'table.txt'
real_out = 'table_out.txt'
fix_html_img(real_in, real_out)

## Importing Data

Luckily, Pandas includes a function that will easily translate HTML code to a dataframe.

In [6]:
df = pd.read_html('table_out.txt')[0]
df = df.fillna('')

In [8]:
#Change columns to a more usable format
df.columns = ['Map', 'Roundtype', 'Blue_Team', 'Red_Team', 'time', 'len', 'KB',
       'KR', 'UB', 'UR', 'FB', 'Death_Sequence', 'Ults_Blue', 'Ults_Red',
       'Teamcomp_Blue', 'Teamcomp_Red']

In [9]:
assert(list(df['Map'].unique()) == ['Dorado', 'Temple of Anubis', 'Ilios', 'Numbani', 'Eichenwalde', 'Junkertown', 'Oasis', 'Horizon Lunar Colony', 'Lijiang Tower'])
assert(list(df['Roundtype'].unique()) == ['Attack', 'Defense', 'Lighthouse', 'Ruins', 'Well', 'University', 'Gardens', 'City Center', 'Control Center', 'Night Market', 'Garden'])
assert(list(df['Blue_Team'].unique()) == ['SFS', 'SHD', 'DAL','LDN', 'PHI', 'BOS','VAL', 'FLA', 'NYE', 'SEO', 'HOU', 'GLA'])
assert(list(df['Red_Team'].unique()) == ['VAL', 'GLA', 'SEO', 'FLA','HOU', 'NYE', 'DAL', 'BOS', 'SHD', 'PHI', 'LDN', 'SFS'])

In [10]:
df['Death_Sequence'] = df['Death_Sequence'].str[2:].str.split(', ')
df['Teamcomp_Blue'] = df['Teamcomp_Blue'].str[2:].str.split(', ')
df['Teamcomp_Red'] = df['Teamcomp_Red'].str[2:].str.split(', ')
df['Ults_Blue'] = df['Ults_Blue'].str[2:].str.split(', ')
df['Ults_Red'] = df['Ults_Red'].str[2:].str.split(', ')

Unfortunately, the dataframe did not record the "winner" of a fight. I will define the winner as the team who has gotten the most kills.

In [12]:
def findWinner(x):
    if x['KR'] > x['KB']:
        return 'Red'
    elif x['KR'] < x['KB']:
        return 'Blue'
    else:
        return 'Tie'
    
df['Winner'] = df.apply(findWinner, axis=1)

## Tidying Data

The dataset we have is not the most useful. Specifically, the Death Sequence and Ultimates Used columns are a string of multiple things used. The main goal now is to change those into usable tidy columns.

In [11]:
#Make a tidy column from the death sequence
df['First_kill'] = df['Death_Sequence'].str[0].str.split().str[1]

In [13]:
assert(list(df['Winner'].unique()) == ['Blue', 'Red', 'Tie'])

First, change each string entry into a list.

In [14]:
for i in range(6):
    strR = 'Red' + str(i)
    strB = 'Blue' + str(i)
    df[strR] = df['Winner']
    df[strB] = df['Winner']
    for j in range(len(df)):
        df.loc[j, strR] = df['Teamcomp_Red'][j][i]
        df.loc[j, strB] = df['Teamcomp_Blue'][j][i]

In [15]:
cols = df.columns
cols1 = np.append(cols[:-11], [cols[-10], cols[-8], cols[-6], cols[-4], cols[-2]])
cols1 = np.append(cols1, [cols[-11], cols[-9], cols[-7], cols[-5] , cols[-3] , cols[-1]])

df = df[cols1]

Make columns for Red/Blue hero 1 - 6

In [16]:
#Takes a while to run
red_comp = pd.DataFrame([])
blue_comp = pd.DataFrame([])
for i in range(len(df)):
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red0']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red1']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red2']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red3']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red4']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red5']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue0']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue1']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue2']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue3']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue4']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue5']] = 1
red_comp = red_comp.fillna(0)
blue_comp = blue_comp.fillna(0)
df = pd.merge(df, red_comp, left_index=True, right_index=True)
df = pd.merge(df, blue_comp, left_index=True, right_index=True)

Future proofing: I know that I will be using these columns to make a model, so I must convert the columns to numbers.

In [17]:
#Make the winner values easier to understand when we get dummies later
df['Winner'] = df['Winner'].map({'Blue':'Blue_Winner', 'Red':'Red_Winner', 'Tie':'Tie'})
df['FB'] = df['FB'].map({'B':1, 'R':0})

In [18]:
map_series = pd.get_dummies(df['Map'])
df = pd.merge(df, map_series, left_index=True, right_index=True)
round_series = pd.get_dummies(df['Roundtype'])
df = pd.merge(df, round_series, left_index=True, right_index=True)
win_series = pd.get_dummies(df['Winner'])
df = pd.merge(df, win_series, left_index=True, right_index=True)
blue_series = pd.get_dummies(df['Blue_Team'], prefix='Blue')
df = pd.merge(df, blue_series, left_index=True, right_index=True)
red_series = pd.get_dummies(df['Red_Team'], prefix='Red')
df = pd.merge(df, red_series, left_index=True, right_index=True)


I have to drop the columns that we updated now.

In [19]:
df = df.drop(columns=['Death_Sequence', 'Ults_Blue',
 'Ults_Red', 'Teamcomp_Blue',
 'Teamcomp_Red',  'Red0',
 'Red1',
 'Red2',
 'Red3',
 'Red4',
 'Red5',
 'Blue0',
 'Blue1',
 'Blue2',
 'Blue3',
 'Blue4',
 'Blue5'])

In [20]:
df.head()

,Map,Roundtype,Blue_Team,Red_Team,time,len,KB,KR,UB,UR,...,Red_FLA,Red_GLA,Red_HOU,Red_LDN,Red_NYE,Red_PHI,Red_SEO,Red_SFS,Red_SHD,Red_VAL
0,Dorado,Attack,SFS,VAL,0:29,14,5,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Dorado,Attack,SFS,VAL,1:05,24,1,6,2,3,...,0,0,0,0,0,0,0,0,0,1
2,Dorado,Attack,SFS,VAL,1:51,7,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
3,Dorado,Attack,SFS,VAL,2:14,28,7,2,3,3,...,0,0,0,0,0,0,0,0,0,1
4,Dorado,Attack,SFS,VAL,3:24,17,4,2,3,1,...,0,0,0,0,0,0,0,0,0,1


## Save the Data!

In [21]:
df.to_csv('/data/cvaranese/OverwatchFightsData.csv')